# Evaluation metrics for ASR

**Levenshtein** mesafesini NLP'den biliyorsanız, konuşma tanıma sistemlerini değerlendirmek için kullanılan ölçütler de size tanıdık gelecektir! Aşina değilseniz endişelenmeyin, farklı ölçümleri bildiğinizden ve ne anlama geldiklerini anladığınızdan emin olmak için açıklamaları baştan sona gözden geçireceğiz.

Konuşma tanıma sistemlerini değerlendirirken, sistemin tahminlerini hedef metin transkripsiyonlarıyla karşılaştırıyor ve mevcut hataları not ediyoruz. Bu hataları üç kategoriden birine ayırıyoruz:

- Yer değiştirmeler (Substitutions) (S): Tahminimizde yanlış kelimeyi transkribe ettiğimiz durumlar ("sat" yerine "sit")
- Eklemeler (Insertions) (I): tahminimize fazladan bir kelime eklediğimiz yer
- Silme (Deletions) (D): Tahminimizdeki bir kelimeyi çıkardığımız yer

Bu hata kategorileri tüm konuşma tanıma metrikleri için aynıdır. Farklı olan, bu hataları hangi seviyede hesapladığımızdır: bunları **kelime seviyesinde** veya **karakter seviyesinde** hesaplayabiliriz.

Metrik tanımlarının her biri için çalışan bir örnek kullanacağız. Burada, bir temel gerçek veya referans metin dizimiz var:

`reference = "the cat sat on the mat"`

`prediction = "the cat sit on the"`

## Word Error Rate

Kelime hata oranı (WER) metriği, konuşma tanıma için 'de facto' metriktir. Kelime düzeyinde değiştirmeleri, eklemeleri ve silmeleri hesaplar. Bu, hataların kelime bazında açıklandığı anlamına gelir. Örneğimizi ele alalım:

| Reference                           | the | cat | sat | on  | the | mat |
|-------------------------------------|-----|-----|-----|-----|-----|-----|
| Prediction                          | the | cat | sit | on  | the |     |
| Label                               | ✅   | ✅   | S   | ✅   | ✅   | D   |

Burada:

- 1 yer değiştirme ("sat" yerine "sit")
- 0 ekleme
- 1 silme ("mat" eksik)

Bu toplamda 2 hata verir. Hata oranımızı elde etmek için, hata sayısını referansımızdaki toplam kelime sayısına (N) böleriz, ki bu örnek için bu sayı 6'dır:

$$WER = \frac{S + I + D}{N} = \frac{1 + 0 + 1}{6} = 0.333
$$

Yani 0,333 veya %33,3'lük bir WER değerimiz var. "Sit" kelimesinin sadece bir karakterinin yanlış olduğuna, ancak kelimenin tamamının yanlış olarak işaretlendiğine dikkat edin. Bu, **WER'in belirleyici bir özelliğidir: yazım hataları ne kadar küçük olursa olsun ağır bir şekilde cezalandırılır**.

WER, düşük olan daha iyi olacak şekilde tanımlanmıştır: daha düşük bir WER, tahminimizde daha az hata olduğu anlamına gelir, bu nedenle mükemmel bir konuşma tanıma sistemi sıfır WER'ye (hata yok) sahip olacaktır.

Şimdi Evaluate kullanarak WER'i nasıl hesaplayabileceğimizi görelim. WER metriğimizi hesaplamak için iki pakete ihtiyacımız olacak: API arayüzü için Evaluate ve hesaplamayı çalıştırmanın ağır işini yapmak için JIWER:

In [1]:
!pip install -q -U evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from evaluate import load

reference = "the cat sat on the mat"
prediction = "the cat sit on the"

wer_metric = load("wer")
wer = wer_metric.compute(references=[reference], predictions=[prediction])

print(wer)

0.3333333333333333


Şimdi, burada oldukça kafa karıştırıcı bir şey var... WER'in üst sınırının ne olduğunu düşünüyorsunuz? Bunun 1 ya da %100 olmasını beklersiniz, değil mi? Hayır! WER, hataların kelime sayısına (N) oranı olduğundan, WER için bir üst sınır yoktur! Örnek olarak 10 kelime tahmin ettiğimizi ve hedefte sadece 2 kelime olduğunu düşünelim. Tüm tahminlerimiz yanlış olsaydı (10 hata), 10 / 2 = 5 veya %500'lük bir WER'e sahip olurduk! Bir ASR sistemini eğitiyorsanız ve %100'ün üzerinde bir WER görüyorsanız bu akılda tutulması gereken bir şeydir. Yine de bunu görüyorsanız, muhtemelen bir şeyler ters gitmiştir... 😅

## Word Accuracy

Daha yüksek olanın daha iyi olduğu bir metrik vermek için WER'i tersine çevirebiliriz. Kelime hata oranını ölçmek yerine, sistemimizin kelime doğruluğunu (WAcc) ölçebiliriz:

$$WAcc = 1 - WER$$

WAcc de kelime düzeyinde ölçülür, sadece WER bir hata metriğinden ziyade bir doğruluk metriği olarak yeniden formüle edilmiştir. WAcc, konuşma literatüründe çok nadiren alıntılanır - sistem tahminlerimizi kelime hataları açısından düşünüyoruz ve bu nedenle bu hata türü ek açıklamalarıyla daha ilişkili olan hata oranı ölçümlerini tercih ediyoruz.

## Character Error Rate

Aslında sadece bir harf yanlışken "sit" kelimesinin tamamını yanlış olarak işaretlememiz biraz haksızlık gibi görünüyor. Bunun nedeni, sistemimizi kelime düzeyinde değerlendiriyor olmamız ve böylece hataları kelime bazında açıklamamızdır. Karakter hata oranı (CER) sistemleri karakter düzeyinde değerlendirir. Bu, kelimelerimizi tek tek karakterlerine ayırdığımız ve hataları karakter bazında not ettiğimiz anlamına gelir:

| Reference                           | t   | h   | e   |   | c   | a   | t   |   | s   | a   | t   |   | o   | n   |   | t   | h   | e   |   | m   | a   | t   |
|-------------------------------------|-----|-----|-----|---|-----|-----|-----|---|-----|-----|-----|---|-----|-----|---|-----|-----|-----|---|-----|-----|-----|
| Prediction                          | t   | h   | e   |   | c   | a   | t   |   | s   | i   | t   |   | o   | n   |   | t   | h   | e   |   |     |     |     |
| Label                               | ✅   | ✅   | ✅   |   | ✅   | ✅   | ✅   |   | ✅   | S   | ✅   |   | ✅   | ✅   |   | ✅   | ✅   | ✅   |   | D   | D   | D   |

Şimdi "sit" kelimesi için "s" ve "t" harflerinin doğru olarak işaretlendiğini görebiliriz. Sadece "i" ikame hatası (S) olarak etiketlenmiştir. Böylece, sistemimizi kısmen doğru tahmin için ödüllendiriyoruz

Örneğimizde, 1 karakter değişimi, 0 ekleme ve 3 silme var. Toplamda 14 karakterimiz var. Yani, bizim CER'imiz:

$$ CER = \frac{S+I+D}{N} = \frac{1+0+3}{14} = 0.286$$

Doğru! CER değerimiz 0,286 veya %28,6'dır. Bunun WER değerimizden ne kadar düşük olduğuna dikkat edin - yazım hatasını çok daha az cezalandırdık.

## Which metric should I use?

Genel olarak WER, konuşma sistemlerini değerlendirmek için CER'den çok daha fazla kullanılmaktadır. Bunun nedeni, WER'in sistemlerin tahminlerin bağlamını daha iyi anlamasını gerektirmesidir. Örneğimizde, "oturmak" yanlış zaman kipindedir. Fiil ile cümlenin zamanı arasındaki ilişkiyi anlayan bir sistem "sat" fiilinin doğru zamanını tahmin edebilirdi. Konuşma sistemlerimizde bu düzeyde bir anlayışı teşvik etmek istiyoruz. Dolayısıyla WER, CER'den daha az affedici olsa da, geliştirmek istediğimiz anlaşılır sistem türlerine daha elverişlidir. Bu nedenle, biz genellikle WER kullanıyoruz ve sizi de kullanmaya teşvik ediyoruz! Ancak, WER kullanmanın mümkün olmadığı durumlar da vardır. Mandarin ve Japonca gibi bazı dillerde 'kelime' kavramı yoktur ve bu nedenle WER anlamsızdır. Burada, CER'i kullanmaya geri dönüyoruz.

Örneğimizde, WER hesaplanırken yalnızca bir cümle kullandık. Gerçek bir sistemi değerlendirirken genellikle birkaç bin cümleden oluşan bir test setinin tamamını kullanırız. Birden fazla cümle üzerinden değerlendirme yaparken, S, I, D ve N'yi tüm cümlelerde toplarız ve ardından yukarıda tanımlanan formüle göre WER'yi hesaplarız. Bu, görünmeyen veriler için WER'in daha iyi bir tahminini verir.

## Normalisation

Bir ASR modelini noktalama işaretleri ve büyük/küçük harf içeren veriler üzerinde eğitirsek, model transkripsiyonlarında büyük/küçük harf ve noktalama işaretlerini tahmin etmeyi öğrenecektir. Bu, modelimizi toplantıları yazıya dökmek veya dikte etmek gibi gerçek konuşma tanıma uygulamaları için kullanmak istediğimizde harikadır, çünkü tahmin edilen transkripsiyonlar, ortografik olarak adlandırılan bir stil olan büyük/küçük harf ve noktalama işaretleriyle tamamen biçimlendirilecektir.

Bununla birlikte, veri kümesini normalleştirerek harf ve noktalama işaretlerini kaldırma seçeneğimiz de vardır. Veri kümesini normalleştirmek konuşma tanıma görevini kolaylaştırır: modelin artık büyük ve küçük harf karakterleri arasında ayrım yapması veya noktalama işaretlerini yalnızca ses verilerinden tahmin etmesi gerekmez (örneğin, noktalı virgül hangi sesi çıkarır?). Bu nedenle, kelime hata oranları doğal olarak daha düşüktür (yani sonuçlar daha iyidir). Whisper makalesi, transkripsiyonların normalleştirilmesinin WER sonuçları üzerindeki ciddi etkisini göstermektedir (bkz. Whisper makalesinin 4.4. Bölümü). Daha düşük WER'ler elde etsek de, model üretim için mutlaka daha iyi değildir. Büyük/küçük harf ve noktalama işaretlerinin olmaması, modelden tahmin edilen metnin okunmasını önemli ölçüde zorlaştırıyor. LibriSpeech veri kümesinden aynı ses örneği üzerinde Wav2Vec2 ve Whisper'ı çalıştırdığımız önceki bölümdeki örneği ele alalım. Wav2Vec2 modeli ne noktalama işaretlerini ne de büyük/küçük harf kullanımını tahmin ederken Whisper her ikisini de tahmin ediyor. Transkripsiyonları yan yana karşılaştırdığımızda, Whisper transkripsiyonunun okunmasının çok daha kolay olduğunu görüyoruz:

```
Wav2Vec2:  HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAUS AND ROSE BEEF LOOMING BEFORE US SIMALYIS DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND
Whisper:   He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind.
```

Whisper transkripsiyonu ortografiktir ve bu nedenle kullanıma hazırdır - hem noktalama işaretleri hem de büyük/küçük harflerle bir toplantı transkripsiyonu veya dikte metni için beklediğimiz gibi biçimlendirilmiştir. Aksine, sonraki uygulamalar için kullanmak istersek, Wav2Vec2 tahminlerimizde noktalama işaretlerini ve büyük/küçük harfleri geri yüklemek için ek işlem sonrası kullanmamız gerekir.

Normalleştirme ile normalleştirmeme arasında mutlu bir orta yol vardır: sistemlerimizi ortografik transkripsiyonlar üzerinde eğitebilir ve ardından WER'i hesaplamadan önce tahminleri ve hedefleri normalleştirebiliriz. Bu şekilde, sistemlerimizi tamamen biçimlendirilmiş metni tahmin edecek şekilde eğitiriz, ancak aynı zamanda transkripsiyonları normalleştirerek elde ettiğimiz WER iyileştirmelerinden de yararlanırız.

Whisper modeli, diğerlerinin yanı sıra büyük/küçük harf, noktalama işaretleri ve sayı biçimlendirmesinin normalleştirilmesini etkin bir şekilde gerçekleştiren bir normalleştirici ile birlikte piyasaya sürülmüştür. Normalleştiriciyi Whisper transkripsiyonlarına uygulayarak bunları nasıl normalleştirebileceğimizi gösterelim:

In [3]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

prediction = " He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind."
normalized_prediction = normalizer(prediction)

normalized_prediction

' he tells us that at this festive season of the year with christmas and roast beef looming before us similarly is drawn from eating and its results occur most readily to the mind '

Metnin tamamen küçük harflerle yazıldığını ve tüm noktalama işaretlerinin kaldırıldığını görebiliriz. Şimdi referans transkripsiyonu tanımlayalım ve ardından referans ile tahmin arasındaki normalleştirilmiş WER'yi hesaplayalım:

In [4]:
reference = "HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND"
normalized_referece = normalizer(reference)

wer = wer_metric.compute(
    references=[normalized_referece], predictions=[normalized_prediction]
)
wer

0.0625

6,25 - LibriSpeech doğrulama setinde Whisper temel modeli için beklediğimiz oran. Burada gördüğümüz gibi, ortografik bir transkripsiyon tahmin ettik, ancak WER'i hesaplamadan önce referans ve tahmini normalleştirerek elde edilen WER artışından yararlandık.

Transkripsiyonları nasıl normalleştireceğinizi seçmek nihayetinde ihtiyaçlarınıza bağlıdır. Her iki dünyanın da en iyisini elde etmek için ortografik metin üzerinde eğitim almanızı ve normalleştirilmiş metin üzerinde değerlendirme yapmanızı öneririz.

## Putting it all together

Pekala! Bu Ünitede şu ana kadar üç konuyu ele aldık: önceden eğitilmiş modeller, veri kümesi seçimi ve değerlendirme. Biraz eğlenelim ve bunları uçtan uca bir örnekte bir araya getirelim 🚀 Önceden eğitilmiş Whisper modelini Common Voice 13 Dhivehi test setinde değerlendirerek kendimizi ince ayarla ilgili bir sonraki bölüme hazırlayacağız. Elde ettiğimiz WER sayısını ince ayar çalışmamız için bir taban çizgisi veya geçmeye çalışacağımız bir hedef sayı olarak kullanacağız 🥊

İlk olarak, pipeline() sınıfını kullanarak önceden eğitilmiş Whisper modelini yükleyeceğiz. Bu süreç şimdiye kadar son derece tanıdık gelecektir! Yapacağımız tek yeni şey, bir GPU üzerinde çalışıyorsa modeli yarı hassas (float16) olarak yüklemektir - bu, WER doğruluğuna neredeyse hiçbir maliyeti olmadan çıkarımı hızlandıracaktır.

In [5]:
import torch
from transformers import pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    torch_dtype=torch.float16,
    device=device,
)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
!pip install -q datasets

In [8]:
from datasets import load_dataset

common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv",
    split="test", trust_remote_code=True
)

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2677it [00:00, 79788.46it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2227it [00:00, 30723.41it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2212it [00:00, 79330.66it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5019it [00:00, 50183.47it/s]
Reading metadata...: 16395it [00:00, 54488.83it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1653it [00:00, 80000.74it/s]


Tüm bir veri kümesi üzerinde değerlendirme yapmak, tek bir örnek üzerinde değerlendirme yapmakla aynı şekilde yapılabilir - tek yapmamız gereken tek bir örnek üzerinde çıkarım yapmak yerine giriş sesleri üzerinde döngü yapmaktır. Bunu yapmak için önce veri setimizi bir KeyDataset'e dönüştürürüz. Bunun tek yaptığı, modele iletmek istediğimiz belirli veri kümesi sütununu seçmek (bizim durumumuzda bu "ses" sütunudur) ve geri kalanını (çıkarım için kullanmak istemediğimiz hedef transkripsiyonlar gibi) göz ardı etmektir. Daha sonra bu dönüştürülmüş veri kümeleri üzerinde yineleme yaparak model çıktılarını tahminleri kaydetmek için bir listeye ekliyoruz. Aşağıdaki kod hücresi, 12 GB belleğe ulaşan yarım hassasiyetli bir GPU üzerinde çalıştırıldığında yaklaşık beş dakika sürecektir:

In [13]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_predictions = []

# run streamed inference
for prediction in tqdm(
    pipe(
        KeyDataset(common_voice_test, "audio"),
        max_new_tokens=128,
        generate_kwargs={"task": "transcribe"},
    ),
    total=len(common_voice_test),
):
    all_predictions.append(prediction["text"])


  0%|          | 0/2212 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 2212/2212 [36:50<00:00,  1.00it/s]


Ve son olarak, WER'i hesaplayabiliriz. İlk olarak ortografik WER'i, yani herhangi bir son işleme tabi tutulmamış WER'i hesaplayalım:

In [14]:
from evaluate import load

wer_metric = load("wer")

wer_ortho = 100 * wer_metric.compute(
    references=common_voice_test["sentence"], predictions=all_predictions
)
wer_ortho

150.86705202312137

Tamam... %150 aslında modelimizin çöp çıktısı verdiği anlamına geliyor. Endişelenmeyin, amacımız Dhivehi eğitim seti üzerinde modele ince ayar yaparak bunu iyileştirmek olacak!

Daha sonra, normalleştirilmiş WER'i, yani işlem sonrası normalleştirme ile WER'i değerlendireceğiz. Normalleştirmeden sonra boş kalacak örnekleri filtrelememiz gerekir, aksi takdirde referansımızdaki (N) toplam kelime sayısı sıfır olur ve bu da hesaplamamızda sıfıra bölme hatası verir:

In [15]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

# compute normalised WER
all_predictions_norm = [normalizer(pred) for pred in all_predictions]
all_references_norm = [normalizer(label) for label in common_voice_test["sentence"]]

# filtering step to only evaluate the samples that correspond to non-zero references
all_predictions_norm = [
    all_predictions_norm[i]
    for i in range(len(all_predictions_norm))
    if len(all_references_norm[i]) > 0
]
all_references_norm = [
    all_references_norm[i]
    for i in range(len(all_references_norm))
    if len(all_references_norm[i]) > 0
]

wer = 100 * wer_metric.compute(
    references=all_references_norm, predictions=all_predictions_norm
)

wer

110.4739715547519

Referanslarımızı ve tahminlerimizi normalleştirerek elde ettiğimiz WER'deki ciddi düşüşü bir kez daha görüyoruz: temel model %150'lik bir ortografik test WER'i elde ederken, normalleştirilmiş WER %110'dır.